# Package Load

In [1]:
import preprocessing
import tensorflow as tf
import numpy as np
import random
import sys
import utils
import pickle

# preprocessing : 이미지 파일로 존재하는 데이터를 numpy와 onehotencoding 라벨로 바꿔주는 모듈
# tensorflow : 텐서플로우 , 일반적으로 tf로 줄여서 사용
# numpy : fundamental package for scientific computing with Python
# pickle : python 의 데이터 형식을 그대로 저장하기 위한 파이썬 기본 패키지


# Image Loading

In [2]:
train_imgs , train_labs , test_imgs , test_labs=preprocessing.get_cifar_data()

def next_batch(imgs , labs , batch_size):
    indices=random.sample(range(len(imgs)) , batch_size)
    batch_xs=imgs[indices]
    batch_ys=labs[indices]
    return batch_xs , batch_ys

# preprocessing 모듈에서 train_imgs , train_labs , test_imgs , test_labs 를 불러 온다.
# fully connected 모델을 위해 32x32x3 크기의 이미지를 3072 으로 reshape 한다.
# next_batch 함수는 batch_size 수 만큼 random 으로 추출하여 train batch 를 생성하는 함수


# Set Placeholder And Variables

In [3]:

height = 32
width = 32
color_ch=3
n_classes = 10
learning_rate=0.001
max_iter=10000
check_point=100
x_ =tf.placeholder(tf.float32, [ None , height , width , color_ch ])
y_ =tf.placeholder( tf.int32 , [ None , n_classes ])

# Placeholder 는 차후에 입력할 값
# Variables 는 weight 로 학습시 변하는 값


# Network Modeling

In [4]:
out_ch=28
w1=tf.get_variable("w1" , [7,7,color_ch , out_ch] , initializer=tf.contrib.layers.xavier_initializer())
b1=tf.Variable(tf.constant(0.1) ,out_ch)
s1=[1,1,1,1]
p1='SAME'
layer1=tf.nn.conv2d(x_ , w1 , s1 , p1 )+b1
layer1=tf.nn.relu(layer1)


out_ch2=64
w2=tf.get_variable("w2" , [5,5,out_ch, out_ch2] , initializer=tf.contrib.layers.xavier_initializer())
b2=tf.Variable(tf.constant(0.1) ,out_ch2)
s2=[1,1,1,1]
layer2=tf.nn.conv2d(layer1, w2 , s2, padding='SAME')+b2
layer2=tf.nn.relu(layer2)
#fully connected layer = affine layer

end_conv_layer=layer2
flatten_layer=tf.contrib.layers.flatten(end_conv_layer)
length=flatten_layer.get_shape()[1]
fc_w1=tf.get_variable("fc_w1" ,[length,n_classes])
fc_b1=tf.Variable(tf.constant(0.1) , n_classes)
y_conv=tf.matmul(flatten_layer ,fc_w1 )+fc_b1

# 2개의 convolution neural network 와 1개의 fully connected_layer 로 구성
# stride : 좌우로 몇 칸 씩 커널을 이동 할 것인지에 대한 값
# padding : convolution 전후로 activation map 의 크기를 조정하기 위한 값 , SAME 을 입력하면 항상 convolution 전후의 크기가 같다.
# weight : [커널의 가로크기, 커널의 세로크기, input activation map 의 채널 크기, output activiation map 의 채널 크기]


# Define Functions

In [5]:
pred=tf.nn.softmax(y_conv)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= pred , labels=y_) , name='cost')
train=tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost,name='train')
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1), name='correct_pred')
accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32), name='accuracy')

# softmax : 결과값을 각 class 에 해당하는 확률값을로 리턴
# softmax_cross_entropy_with_logits : 실제 정답인 onehot vector 와 예측값 pred 를 차이를 cross_entropy 로 계산
# tf.train.GradientDescentOptimizer : cost 가 최소와 되도록 weight를 조정하는 함수
# accuracy : 실제 값과 예측값의 일치률

# Training

In [6]:
sess=tf.Session()
init=tf.global_variables_initializer()
sess.run(init)
saver = tf.train.Saver()

batch_iteration = 100    
training_epochs = 2000

train_cost_list = []
test_cost_list = []
train_acc_list = []
test_acc_list = []

# Tensorflow 학습을 위한 session 정의
# saver : 학습한 weight 와 graph 를 저장

# sess.run 을 통해 함수를 실행 , feed_dict 는 placeholder 에 들어갈 값


for epoch in range(training_epochs):
    
    train_avg_cost = 0.
    train_avg_acc = 0.
    test_avg_cost = 0.
    test_avg_acc = 0.
    
    for batch in range(batch_iteration) :
        batch_xs , batch_ys = next_batch(train_imgs , train_labs , 60)
        
        sess.run(train, feed_dict={x_:batch_xs , y_ : batch_ys})
        train_avg_cost += sess.run(cost, feed_dict={x_:batch_xs , y_ : batch_ys})
        train_avg_acc += sess.run(accuracy, feed_dict={x_:batch_xs , y_ : batch_ys})
        
    train_avg_cost = train_avg_cost / batch_iteration
    train_avg_acc = train_avg_acc / batch_iteration
    
    test_avg_cost = sess.run(cost, feed_dict={x_: test_imgs, y_: test_labs})
    test_avg_acc = sess.run(accuracy, feed_dict={x_: test_imgs, y_: test_labs})
    
    
    print "##################################"
    print ("Epoch: %03d/%03d train cost: %.4f" % (epoch, training_epochs, train_avg_cost))
    print ("Epoch: %03d/%03d train acc: %.4f" % (epoch, training_epochs, train_avg_acc))
    print ("Epoch: %03d/%03d test cost: %.4f" % (epoch, training_epochs, test_avg_cost))
    print ("Epoch: %03d/%03d test acc: %.4f" % (epoch, training_epochs, test_avg_acc))    
    
    train_cost_list.append(train_avg_cost)
    test_cost_list.append(test_avg_cost)
    train_acc_list.append(train_avg_acc)
    test_acc_list.append(train_avg_acc)
   
    
    
saver.save(sess, "model/cifar_convolution.ckpt")

with open('cost_acc/cifar_convolution_train_cost', 'wb') as fp:
    pickle.dump(train_avg_cost, fp)
with open('cost_acc/cifar_convolution_test_cost', 'wb') as fp:
    pickle.dump(test_avg_cost, fp)
with open('cost_acc/cifar_convolution_train_acc', 'wb') as fp:
    pickle.dump(train_avg_acc, fp)
with open('cost_acc/cifar_convolution_test_acc', 'wb') as fp:
    pickle.dump(test_avg_acc, fp)
    

##################################
Epoch: 000/2000 train cost: 2.3016
Epoch: 000/2000 train acc: 0.1138
Epoch: 000/2000 test cost: 2.3017
Epoch: 000/2000 test acc: 0.1037
##################################
Epoch: 001/2000 train cost: 2.3006
Epoch: 001/2000 train acc: 0.1030
Epoch: 001/2000 test cost: 2.3006
Epoch: 001/2000 test acc: 0.1025
##################################
Epoch: 002/2000 train cost: 2.2988
Epoch: 002/2000 train acc: 0.1037
Epoch: 002/2000 test cost: 2.2997
Epoch: 002/2000 test acc: 0.1012
##################################
Epoch: 003/2000 train cost: 2.2968
Epoch: 003/2000 train acc: 0.1080
Epoch: 003/2000 test cost: 2.2983
Epoch: 003/2000 test acc: 0.1011
##################################
Epoch: 004/2000 train cost: 2.2969
Epoch: 004/2000 train acc: 0.0992
Epoch: 004/2000 test cost: 2.2967
Epoch: 004/2000 test acc: 0.1041
##################################
Epoch: 005/2000 train cost: 2.2962
Epoch: 005/2000 train acc: 0.1027
Epoch: 005/2000 test cost: 2.2955
Epoch: 

##################################
Epoch: 048/2000 train cost: 2.1557
Epoch: 048/2000 train acc: 0.3123
Epoch: 048/2000 test cost: 2.1656
Epoch: 048/2000 test acc: 0.2923
##################################
Epoch: 049/2000 train cost: 2.1490
Epoch: 049/2000 train acc: 0.3203
Epoch: 049/2000 test cost: 2.1654
Epoch: 049/2000 test acc: 0.2937
##################################
Epoch: 050/2000 train cost: 2.1492
Epoch: 050/2000 train acc: 0.3190
Epoch: 050/2000 test cost: 2.1640
Epoch: 050/2000 test acc: 0.2976
##################################
Epoch: 051/2000 train cost: 2.1479
Epoch: 051/2000 train acc: 0.3205
Epoch: 051/2000 test cost: 2.1627
Epoch: 051/2000 test acc: 0.2970
##################################
Epoch: 052/2000 train cost: 2.1456
Epoch: 052/2000 train acc: 0.3223
Epoch: 052/2000 test cost: 2.1629
Epoch: 052/2000 test acc: 0.2966
##################################
Epoch: 053/2000 train cost: 2.1410
Epoch: 053/2000 train acc: 0.3255
Epoch: 053/2000 test cost: 2.1651
Epoch: 

##################################
Epoch: 096/2000 train cost: 2.1105
Epoch: 096/2000 train acc: 0.3582
Epoch: 096/2000 test cost: 2.1364
Epoch: 096/2000 test acc: 0.3204
##################################
Epoch: 097/2000 train cost: 2.1108
Epoch: 097/2000 train acc: 0.3540
Epoch: 097/2000 test cost: 2.1350
Epoch: 097/2000 test acc: 0.3237
##################################
Epoch: 098/2000 train cost: 2.1115
Epoch: 098/2000 train acc: 0.3548
Epoch: 098/2000 test cost: 2.1382
Epoch: 098/2000 test acc: 0.3178
##################################
Epoch: 099/2000 train cost: 2.1212
Epoch: 099/2000 train acc: 0.3372
Epoch: 099/2000 test cost: 2.1349
Epoch: 099/2000 test acc: 0.3191
##################################
Epoch: 100/2000 train cost: 2.1172
Epoch: 100/2000 train acc: 0.3467
Epoch: 100/2000 test cost: 2.1330
Epoch: 100/2000 test acc: 0.3240
##################################
Epoch: 101/2000 train cost: 2.1048
Epoch: 101/2000 train acc: 0.3658
Epoch: 101/2000 test cost: 2.1331
Epoch: 

##################################
Epoch: 144/2000 train cost: 2.0981
Epoch: 144/2000 train acc: 0.3670
Epoch: 144/2000 test cost: 2.1298
Epoch: 144/2000 test acc: 0.3257
##################################
Epoch: 145/2000 train cost: 2.0921
Epoch: 145/2000 train acc: 0.3737
Epoch: 145/2000 test cost: 2.1261
Epoch: 145/2000 test acc: 0.3280
##################################
Epoch: 146/2000 train cost: 2.0958
Epoch: 146/2000 train acc: 0.3702
Epoch: 146/2000 test cost: 2.1240
Epoch: 146/2000 test acc: 0.3354
##################################
Epoch: 147/2000 train cost: 2.0923
Epoch: 147/2000 train acc: 0.3747
Epoch: 147/2000 test cost: 2.1239
Epoch: 147/2000 test acc: 0.3358
##################################
Epoch: 148/2000 train cost: 2.0960
Epoch: 148/2000 train acc: 0.3712
Epoch: 148/2000 test cost: 2.1239
Epoch: 148/2000 test acc: 0.3319
##################################
Epoch: 149/2000 train cost: 2.1002
Epoch: 149/2000 train acc: 0.3638
Epoch: 149/2000 test cost: 2.1231
Epoch: 

##################################
Epoch: 192/2000 train cost: 2.0931
Epoch: 192/2000 train acc: 0.3745
Epoch: 192/2000 test cost: 2.1166
Epoch: 192/2000 test acc: 0.3403
##################################
Epoch: 193/2000 train cost: 2.0839
Epoch: 193/2000 train acc: 0.3815
Epoch: 193/2000 test cost: 2.1209
Epoch: 193/2000 test acc: 0.3346
##################################
Epoch: 194/2000 train cost: 2.0918
Epoch: 194/2000 train acc: 0.3713
Epoch: 194/2000 test cost: 2.1138
Epoch: 194/2000 test acc: 0.3433
##################################
Epoch: 195/2000 train cost: 2.0803
Epoch: 195/2000 train acc: 0.3862
Epoch: 195/2000 test cost: 2.1190
Epoch: 195/2000 test acc: 0.3382
##################################
Epoch: 196/2000 train cost: 2.0886
Epoch: 196/2000 train acc: 0.3767
Epoch: 196/2000 test cost: 2.1159
Epoch: 196/2000 test acc: 0.3390
##################################
Epoch: 197/2000 train cost: 2.0834
Epoch: 197/2000 train acc: 0.3818
Epoch: 197/2000 test cost: 2.1150
Epoch: 

...

##################################
Epoch: 1807/2000 train cost: 1.9087
Epoch: 1807/2000 train acc: 0.5668
Epoch: 1807/2000 test cost: 1.9760
Epoch: 1807/2000 test acc: 0.4831
##################################
Epoch: 1808/2000 train cost: 1.9080
Epoch: 1808/2000 train acc: 0.5680
Epoch: 1808/2000 test cost: 1.9785
Epoch: 1808/2000 test acc: 0.4794
##################################
Epoch: 1809/2000 train cost: 1.9080
Epoch: 1809/2000 train acc: 0.5655
Epoch: 1809/2000 test cost: 1.9776
Epoch: 1809/2000 test acc: 0.4805
##################################
Epoch: 1810/2000 train cost: 1.9231
Epoch: 1810/2000 train acc: 0.5492
Epoch: 1810/2000 test cost: 1.9796
Epoch: 1810/2000 test acc: 0.4796
##################################
Epoch: 1811/2000 train cost: 1.9123
Epoch: 1811/2000 train acc: 0.5612
Epoch: 1811/2000 test cost: 1.9772
Epoch: 1811/2000 test acc: 0.4830
##################################
Epoch: 1812/2000 train cost: 1.9157
Epoch: 1812/2000 train acc: 0.5587
Epoch: 1812/2000 te

##################################
Epoch: 1854/2000 train cost: 1.9065
Epoch: 1854/2000 train acc: 0.5687
Epoch: 1854/2000 test cost: 1.9777
Epoch: 1854/2000 test acc: 0.4794
##################################
Epoch: 1855/2000 train cost: 1.9141
Epoch: 1855/2000 train acc: 0.5590
Epoch: 1855/2000 test cost: 1.9765
Epoch: 1855/2000 test acc: 0.4840
##################################
Epoch: 1856/2000 train cost: 1.9097
Epoch: 1856/2000 train acc: 0.5655
Epoch: 1856/2000 test cost: 1.9767
Epoch: 1856/2000 test acc: 0.4808
##################################
Epoch: 1857/2000 train cost: 1.9111
Epoch: 1857/2000 train acc: 0.5658
Epoch: 1857/2000 test cost: 1.9772
Epoch: 1857/2000 test acc: 0.4816
##################################
Epoch: 1858/2000 train cost: 1.9102
Epoch: 1858/2000 train acc: 0.5670
Epoch: 1858/2000 test cost: 1.9810
Epoch: 1858/2000 test acc: 0.4781
##################################
Epoch: 1859/2000 train cost: 1.8927
Epoch: 1859/2000 train acc: 0.5830
Epoch: 1859/2000 te

##################################
Epoch: 1901/2000 train cost: 1.9016
Epoch: 1901/2000 train acc: 0.5717
Epoch: 1901/2000 test cost: 1.9741
Epoch: 1901/2000 test acc: 0.4837
##################################
Epoch: 1902/2000 train cost: 1.9086
Epoch: 1902/2000 train acc: 0.5645
Epoch: 1902/2000 test cost: 1.9756
Epoch: 1902/2000 test acc: 0.4845
##################################
Epoch: 1903/2000 train cost: 1.9057
Epoch: 1903/2000 train acc: 0.5682
Epoch: 1903/2000 test cost: 1.9762
Epoch: 1903/2000 test acc: 0.4829
##################################
Epoch: 1904/2000 train cost: 1.8983
Epoch: 1904/2000 train acc: 0.5763
Epoch: 1904/2000 test cost: 1.9741
Epoch: 1904/2000 test acc: 0.4838
##################################
Epoch: 1905/2000 train cost: 1.9077
Epoch: 1905/2000 train acc: 0.5668
Epoch: 1905/2000 test cost: 1.9761
Epoch: 1905/2000 test acc: 0.4828
##################################
Epoch: 1906/2000 train cost: 1.9046
Epoch: 1906/2000 train acc: 0.5677
Epoch: 1906/2000 te

##################################
Epoch: 1948/2000 train cost: 1.9031
Epoch: 1948/2000 train acc: 0.5735
Epoch: 1948/2000 test cost: 1.9768
Epoch: 1948/2000 test acc: 0.4811
##################################
Epoch: 1949/2000 train cost: 1.8956
Epoch: 1949/2000 train acc: 0.5775
Epoch: 1949/2000 test cost: 1.9736
Epoch: 1949/2000 test acc: 0.4857
##################################
Epoch: 1950/2000 train cost: 1.9074
Epoch: 1950/2000 train acc: 0.5663
Epoch: 1950/2000 test cost: 1.9746
Epoch: 1950/2000 test acc: 0.4830
##################################
Epoch: 1951/2000 train cost: 1.9002
Epoch: 1951/2000 train acc: 0.5753
Epoch: 1951/2000 test cost: 1.9740
Epoch: 1951/2000 test acc: 0.4830
##################################
Epoch: 1952/2000 train cost: 1.9038
Epoch: 1952/2000 train acc: 0.5692
Epoch: 1952/2000 test cost: 1.9754
Epoch: 1952/2000 test acc: 0.4819
##################################
Epoch: 1953/2000 train cost: 1.8961
Epoch: 1953/2000 train acc: 0.5792
Epoch: 1953/2000 te

##################################
Epoch: 1995/2000 train cost: 1.8945
Epoch: 1995/2000 train acc: 0.5802
Epoch: 1995/2000 test cost: 1.9729
Epoch: 1995/2000 test acc: 0.4846
##################################
Epoch: 1996/2000 train cost: 1.8897
Epoch: 1996/2000 train acc: 0.5853
Epoch: 1996/2000 test cost: 1.9723
Epoch: 1996/2000 test acc: 0.4858
##################################
Epoch: 1997/2000 train cost: 1.8915
Epoch: 1997/2000 train acc: 0.5832
Epoch: 1997/2000 test cost: 1.9734
Epoch: 1997/2000 test acc: 0.4859
##################################
Epoch: 1998/2000 train cost: 1.8902
Epoch: 1998/2000 train acc: 0.5853
Epoch: 1998/2000 test cost: 1.9768
Epoch: 1998/2000 test acc: 0.4808
##################################
Epoch: 1999/2000 train cost: 1.9033
Epoch: 1999/2000 train acc: 0.5707
Epoch: 1999/2000 test cost: 1.9780
Epoch: 1999/2000 test acc: 0.4802
